In [ ]:
import sys
import os
from icecream import ic

from pathlib import Path

import utils_behavior

from utils_behavior import Ballpushing_utils
from utils_behavior import Utils
from utils_behavior import Processing
from utils_behavior import HoloviewsTemplates

import pandas as pd
import hvplot.pandas
import numpy as np


from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pandas as pd
import pyarrow.feather as feather

import pandas as pd
import numpy as np
import holoviews as hv
import hvplot.pandas



import matplotlib.pyplot as plt
import seaborn as sns



import holoviews as hv


hv.extension("bokeh")

In [ ]:
# Get the path to the data

Datapath = Utils.get_data_path()

In [ ]:
# Find folders with "Learning or learning" in the name as a list

folders = [f for f in Datapath.glob("*FeedingState*")]

folders

In [ ]:
# Take the first 

exp = Ballpushing_utils.Experiment(folders[0])

In [ ]:
Dataset = Ballpushing_utils.Dataset(exp)

In [ ]:
Dataset.generate_dataset(success_cutoff=False)

In [ ]:
Dataset.data.columns

In [ ]:
# Plot the yball_relative as a function of time grouped by individual flies

# curves = hv.Curve(Dataset.data, kdims=["time"], vdims=["yball_relative","fly"]).groupby("fly").overlay()

# curves.opts(width=800, height=400)

In [ ]:
from holoviews import opts
from bokeh.palettes import viridis

Tries = 30

NbFlies = 7

for i in range(Tries):

    # Pick random flies and plot their yball_relative as a function of time
    flies = np.random.choice(Dataset.data.fly.unique(), NbFlies)

    # Generate a Viridis palette with adapted number of colors colors
    viridis_palette = viridis(NbFlies)

    # Create the curves and overlay them
    curves = (
        hv.Curve(
            Dataset.data[Dataset.data.fly.isin(flies)],
            kdims=["time"],
            vdims=["yball_relative", "fly"],
        )
        .groupby("fly")
        .overlay()
    )

    curves = curves.opts(
        opts.Curve(
            width=800,
            height=400,
            show_grid=True,
            show_legend=False,
            ylabel="ball position relative to start (px)",
            xlabel="time (s)",
            
        )
    )

    # Apply the viridis colormap
    curves = curves.opts(opts.Curve(color=hv.Cycle(values=list(viridis_palette))))

    hv.save(curves, f"/mnt/upramdya_files/DURRIEU_Matthias/DlsImgs/testcurves{i}.png")